In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from lyricsgenius import Genius
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Check if TensorFlow is using the GPU/TPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num TPUs Available: ", len(tf.config.experimental.list_physical_devices('TPU')))

Num GPUs Available:  1
Num TPUs Available:  0


# Lyrics Fetching

In [ ]:
token = os.getenv('GENIUS_ACCESS_TOKEN')
genius = Genius(token)
genius.remove_section_headers = True

In [ ]:
def fetch_lyrics_from_artists(artists, max_songs=10):
    all_lyrics = []
    for artist in artists:
        try:
            artist = genius.search_artist(artist, max_songs=max_songs, sort="popularity")
            for song in artist.songs:
                lyrics = song.lyrics
                cleaned_lyrics = lyrics.split('Lyrics\n', 1)[1] # remove first section before Lyrics
                all_lyrics.append(cleaned_lyrics)
            # Wait to avoid hitting API rate limits
            time.sleep(5)
        except Exception as e:
            print(f"Error fetching lyrics for {artist}: {e}")
            pass
    return all_lyrics

In [ ]:
artists = ['My Chemical Romance', '5 Seconds of Summer', 'Paramore']

lyrics_list = fetch_lyrics_from_artists(artists)

# Preprocessing

In [7]:
def preprocess_lyrics(lyrics_list, max_seq_len=300):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lyrics_list) # generate word_index vocab
    vocab_size = len(tokenizer.word_index) + 1 # +1 because we want 0 indexing

    input_seqs = []
    labels = []
    for lyrics in lyrics_list:
        token_list = tokenizer.texts_to_sequences([lyrics])[0]
        # generate features/target based on ngrams
        for i in range(1, len(token_list)):
            n_gram_seq = token_list[:i+1]
            input_seqs.append(n_gram_seq[:-1])
            labels.append(n_gram_seq[-1])
    input_seqs = pad_sequences(input_seqs, maxlen=max_seq_len-1, padding="pre")
    labels = to_categorical(labels, num_classes = vocab_size)
    return input_seqs, labels, vocab_size, tokenizer

In [8]:
max_seq_len = 300
X, y, vocab_size, tokenizer = preprocess_lyrics(lyrics_list, max_seq_len)

In [9]:
print(X.shape, y.shape)

(19340, 299) (19340, 1115)


In [10]:
# top 10 common words in lyrics
list(tokenizer.word_index.items())[:10]

[('you', 1),
 ('i', 2),
 ('the', 3),
 ('and', 4),
 ('to', 5),
 ('me', 6),
 ('it', 7),
 ('your', 8),
 ('all', 9),
 ('my', 10)]

# RNN

In [11]:
def build_model(vocab_size):
    # Model architecture
    model = models.Sequential([
        layers.Embedding(vocab_size, 128),
        layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
        layers.Dropout(0.2), # Regularization to prevent overfitting
        layers.Bidirectional(layers.LSTM(128)),
        layers.Dense(vocab_size, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Summarize the model architecture
    model.summary()

    return model

In [12]:
model = build_model(vocab_size)
hist = model.fit(X, y, epochs=30, batch_size=64, verbose=2)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
303/303 - 27s - 88ms/step - accuracy: 0.0537 - loss: 5.6854
Epoch 2/30
303/303 - 18s - 60ms/step - accuracy: 0.0711 - loss: 5.1845
Epoch 3/30
303/303 - 21s - 69ms/step - accuracy: 0.0973 - loss: 4.8302
Epoch 4/30
303/303 - 18s - 61ms/step - accuracy: 0.1265 - loss: 4.5433
Epoch 5/30
303/303 - 19s - 61ms/step - accuracy: 0.1608 - loss: 4.2561
Epoch 6/30
303/303 - 21s - 69ms/step - accuracy: 0.1973 - loss: 4.0270
Epoch 7/30
303/303 - 21s - 68ms/step - accuracy: 0.2339 - loss: 3.7831
Epoch 8/30
303/303 - 19s - 62ms/step - accuracy: 0.2802 - loss: 3.4907
Epoch 9/30
303/303 - 20s - 67ms/step - accuracy: 0.3009 - loss: 3.3354
Epoch 10/30
303/303 - 21s - 68ms/step - accuracy: 0.3571 - loss: 3.0573
Epoch 11/30
303/303 - 19s - 63ms/step - accuracy: 0.3871 - loss: 2.8671
Epoch 12/30
303/303 - 20s - 67ms/step - accuracy: 0.3979 - loss: 2.7814
Epoch 13/30
303/303 - 19s - 62ms/step - accuracy: 0.4340 - loss: 2.5940
Epoch 14/30
303/303 - 19s - 62ms/step - accuracy: 0.4622 - loss: 2.4594
E

In [13]:
def generate_text(seed_text, max_seq_len, model, tokenizer, next_words=70, newline_every=7):
    word_count = len(seed_text.split())
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding="pre")
        predicted = model.predict(token_list, verbose=0)
        output_word = tokenizer.index_word[np.argmax(predicted)]
        sep = "\n"  if word_count % newline_every == 0 else " "
        seed_text += sep + output_word
        word_count += 1
    return seed_text

In [14]:
seed_text = "In the middle of the scary night"
generated_lyrics = generate_text(seed_text, max_seq_len, model, tokenizer)
print(generated_lyrics)

In the middle of the scary night
believers the river in the morning rays
and as the hardest part of all
our bark when you are the hardest
part of this is leaving you now
as a relic to be pure against
was your clothes or strike a violent
pose maybe they'll leave you alone but
not me so long and goodnight so
darken your clothes or strike a violent
pose maybe they'll leave you alone but


In [15]:
model_dir = "../../models"
model.save(f"{model_dir}/lyrics_gen_model.keras")